In [53]:
# Cell 1: Importing necessary libraries
from datasets import load_dataset
import string
from tqdm import tqdm
from rank_bm25 import BM25Okapi
import pandas as pd
import json
from underthesea import word_tokenize
from sklearn.metrics import recall_score, f1_score
# Stopwords tiếng Việt
stop_words_vn = set([
    "và", "của", "là", "các", "trong", "với", "cho", "để", "những", "khi", 
    "thì", "này", "làm", "từ", "đã", "sẽ", "rằng", "mà", "như", "lại", 
    "ra", "sau", "cũng", "vậy", "nếu", "đến", "thế", "biết", "theo", "đâu", 
    "đó", "trước", "vừa", "rồi", "trên", "dưới", "ngoài", "gì", "còn", "nữa", 
    "nào", "hết", "ai", "ấy", "lúc", "ở", "đi", "về", "ngay", "luôn", "đang", 
    "thì", "đây", "kia", "ấy", "điều", "việc", "vì", "giữa", "qua", "vẫn", 
    "chỉ", "nói", "thật", "hơn", "vậy", "hay", "lại", "ngày", "giờ", "tại", 
    "bởi", "sao", "trước", "sau", "đó", "mà", "về", "đến", "thì", "được", 
    "thế", "còn", "đến", "cũng", "này", "đấy", "một", "vì", "những", "thì", 
    "vậy", "thế", "đây", "vẫn", "lại", "thì", "còn", "đó", "này", "ở", 
    "trong", "làm", "khi", "vậy", "này", "đó", "ở", "được", "làm", "để", 
    "khi", "với", "về", "đi", "cho", "về", "đã", "với", "như", "đi", "này", 
    "như", "được", "cho", "thì", "làm", "ở", "như", "điều", "khi", "với",
    "trong", "!", ".", ",", ":", ";", "?", "(", ")", "[", "]", "{", "}", "..."
])


In [54]:
# Cell 2: Loading the dataset
# File này chứa các đoạn văn bản (chunks)
chunks_filepath = "/Users/nhotin/Documents/GitHub/LegalBizAI_project/data/testset/id_cof/chunk_sz_fl_point/all_chunk_final.json"
with open(chunks_filepath, "r", encoding="utf-8") as f:
    chunks = json.load(f)

# File này chứa các câu hỏi và id của các chunks liên quan
qa_filepath = "/Users/nhotin/Documents/GitHub/LegalBizAI_project/data/testset/id_cof/chunk_sz_fl_point/quaset_final.json"
df = pd.read_json(qa_filepath)[["question", "chunk_ids"]]

In [55]:
# Cell 3: Function to split text (cải tiến)
def split_text(text):
    # Loại bỏ dấu câu và các ký tự đặc biệt, sau đó tách từ
    text = re.sub(r'[^\w\s]', '', ud.normalize('NFC', text)).lower()
    words = word_tokenize(text, format="text").split() 
    return [word for word in words if word not in stop_words_vn]


In [56]:
# Cell 4: Function to retrieve relevant chunks using BM25 (cải tiến)
def retrieve(question, topk=3):
    tokenized_query = split_text(question)
    bm25_scores = bm25.get_scores(tokenized_query)
    top_n = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)[:topk]
    return [chunks[i] for i in top_n]

In [57]:
# Cell 5: Initiate BM25 retriever with parameter tuning (cải tiến)
tokenized_corpus = [split_text(doc["passage"]) for doc in tqdm(chunks)]
bm25 = BM25Okapi(tokenized_corpus)

100%|██████████| 4162/4162 [00:12<00:00, 321.38it/s]


In [59]:
# Cell 6: Function to get the full article passage
def get_full_article(chunk_ids: list[int]) -> dict:
    articles_ids = set()
    for chunk_id in chunk_ids:
        if chunk_id in articles_ids:
            continue
        articles_ids.add(chunk_id)
        chunk_title = chunks[chunk_id]["title"]
        run_id = chunk_id - 1
        while run_id >= 0 and chunks[run_id]["title"] == chunk_title:
            articles_ids.add(run_id)
            run_id -= 1
        run_id = chunk_id + 1
        while run_id < len(chunks) and chunks[run_id]["title"] == chunk_title:
            articles_ids.add(run_id)
            run_id += 1
    articles_ids = sorted(articles_ids)
    content_lines = []
    chunk_title = ""
    for id in articles_ids:
        if chunk_title != chunks[id]["title"]:
            chunk_title = chunks[id]["title"]
            content_lines.append(chunk_title)
        passage_lines = chunks[id]["passage"].splitlines()
        content_lines.extend(passage_lines[1:])
    content = "\n".join(content_lines)
    return {"ids": articles_ids, "content": content}


In [60]:
# Cell 7: Function to calculate F1 beta score
def f1_beta(pred, actual, beta=4):
    TP, FP, FN = 0, 0, 0
    for pred_list, actual_list in zip(pred, actual):
        pred_set = set(pred_list)
        actual_set = set(actual_list)
        TP += len(pred_set & actual_set)
        FP += len(pred_set - actual_set)
        FN += len(actual_set - pred_set)
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    if precision + recall > 0:
        f1_beta_score = (1 + beta**2) * (precision * recall) / (beta**2 * precision + recall)
    else:
        f1_beta_score = 0
    score = dict()
    score["recall"] = recall
    score["precision"] = precision
    score["f1_beta"] = f1_beta_score
    return score


In [61]:
# Cell 8: Function to get retrieval ids for a question (cải tiến)
def retrieval_ids(question):
    top_chunks = retrieve(question, topk=3)
    return get_full_article([chunk["id"] for chunk in top_chunks])["ids"]



In [62]:
# Cell 9: Apply retrieval_ids function and evaluate the model with parameter tuning
best_recall = 0
best_f1_beta = 0
best_params = (None, None)

k1_values = [1.2, 1.5, 1.8, 2.0]  
b_values = [0.5, 0.6, 0.75, 0.9]    
for k1 in k1_values:
    for b in b_values:
        bm25.k1 = k1
        bm25.b = b
        df["pred_ids"] = df["question"].apply(retrieval_ids)
        actual = df["chunk_ids"].tolist()
        pred = df["pred_ids"].to_list()
        scores = f1_beta(pred, actual)
        if scores["recall"] > best_recall and scores["f1_beta"] > best_f1_beta:
            best_recall = scores["recall"]
            best_f1_beta = scores["f1_beta"]
            best_params = (k1, b)

print(f"Best recall: {best_recall:.4f}")
print(f"Best f1_beta: {best_f1_beta:.4f}")
print(f"Best parameters: k1 = {best_params[0]}, b = {best_params[1]}")

bm25 = BM25Okapi(tokenized_corpus, k1=best_params[0], b=best_params[1])

Best recall: 0.7192
Best f1_beta: 0.6771
Best parameters: k1 = 1.8, b = 0.9


In [63]:
# Cell 10: Display the dataframe and compare predictions with actual chunk ids
df[df["chunk_ids"] == df["pred_ids"]]

,question,chunk_ids,pred_ids
9,Hội đồng giải thể doanh nghiệp do Nhà nước nắm...,"[2799, 2800, 2801, 2802, 2803, 2804, 2805, 280...","[2799, 2800, 2801, 2802, 2803, 2804, 2805, 280..."
11,Doanh nghiệp do Nhà nước nắm giữ 100% vốn điều...,"[2763, 2764, 2765, 2766, 2767, 2768, 2769, 277...","[2763, 2764, 2765, 2766, 2767, 2768, 2769, 277..."
15,Kể từ khi có quyết định giải thể doanh nghiệp ...,"[1715, 1716, 1717, 1718, 1719, 1720, 1721, 172...","[1715, 1716, 1717, 1718, 1719, 1720, 1721, 172..."
19,"Kể từ khi có quyết định giải thể doanh nghiệp,...","[1715, 1716, 1717, 1718, 1719, 1720, 1721, 172...","[1715, 1716, 1717, 1718, 1719, 1720, 1721, 172..."
27,Doanh nghiệp không được thực hiện những hoạt đ...,"[1715, 1716, 1717, 1718, 1719, 1720, 1721, 172...","[1715, 1716, 1717, 1718, 1719, 1720, 1721, 172..."
...,...,...,...
2461,Cổ đông phổ thông có quyền yêu cầu triệu tập h...,"[902, 903, 904, 905, 906, 907, 908, 909, 910, ...","[902, 903, 904, 905, 906, 907, 908, 909, 910, ..."
2462,Thời hạn Hội đồng quản trị phải triệu tập họp ...,"[1083, 1084, 1085, 1086, 1087, 1088, 1089, 109...","[1083, 1084, 1085, 1086, 1087, 1088, 1089, 109..."
2465,Chủ tịch Hội đồng quản trị triệu tập họp Hội đ...,"[1270, 1271, 1272, 1273, 1274, 1275, 1276, 127...","[1270, 1271, 1272, 1273, 1274, 1275, 1276, 127..."
2467,Thời hiệu xử phạt vi phạm hành chính đối với h...,"[3081, 3082, 3083, 3084, 3085]","[3081, 3082, 3083, 3084, 3085]"
